**All Rounders Regression**

Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import RepeatedKFold


Mounting Google Drive to the Notebook

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading the Dataset

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/ALL_ROUNDERS_AVG_POINTS.csv")

In [4]:
df

,all_rounder,match_id,batted_for,opposition_team,venue,bat_inning,batsman_runs,balls,strike_rate,total_batting_points,bat_fp,total_points,batting_avg_inning_points,batting_avg_venue_points,batting_avg_oppo_points,avg_strike_rate,avg_batsman_runs,bowl_inning,total_bowler_runs,maiden,economy,overs,wickets,bowling_fp,bowl_fp_avg,bowl_opp_avg,bowl_ven_avg,bowl_inn_avg,bowl_run_avg,eco_avg,maiden_avg,wicket_avg,bowl_over_avg,rating
0,A Ashish Reddy,286,Deccan Chargers,Mumbai Indians,Wankhede Stadium,1,10,10,100.0,7,0,7,7.0,7.000000,7.0,100.0,10.00,2,11,0,3.666667,3,1,31,58.000000,31.000000,31.00,44.500000,21.500000,5.833333,0.000000,1.500000,3.500000,8.0
1,A Ashish Reddy,292,Deccan Chargers,Chennai Super Kings,"MA Chidambaram Stadium, Chepauk",2,3,3,100.0,3,0,3,3.0,3.000000,3.0,100.0,6.50,1,16,0,8.000000,2,1,25,37.333333,25.000000,25.00,25.000000,22.750000,7.583333,0.000000,1.250000,3.000000,8.0
2,A Ashish Reddy,298,Deccan Chargers,Kings XI Punjab,"Rajiv Gandhi International Stadium, Uppal",2,8,8,100.0,5,0,5,4.0,5.000000,5.0,100.0,7.00,1,39,0,9.750000,4,2,72,27.000000,72.000000,72.00,48.500000,27.666667,8.180556,0.000000,1.333333,3.333333,8.0
3,A Ashish Reddy,314,Deccan Chargers,Rajasthan Royals,"Rajiv Gandhi International Stadium, Uppal",2,10,4,250.0,4,0,4,4.0,4.500000,4.0,137.5,7.75,1,17,0,8.500000,2,0,0,33.000000,0.000000,22.00,32.333333,26.500000,9.010417,0.000000,1.000000,3.000000,9.5
4,A Ashish Reddy,317,Deccan Chargers,Royal Challengers Bangalore,"Rajiv Gandhi International Stadium, Uppal",1,4,5,80.0,4,0,4,5.5,4.333333,4.0,126.0,7.00,2,25,0,6.250000,4,3,81,22.000000,57.000000,36.75,36.666667,26.333333,8.703704,0.000000,1.222222,3.111111,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496,I Udana,784,0,Kolkata Knight Riders,Sharjah Cricket Stadium,0,0,0,0.0,0,0,0,0.0,0.000000,0.0,0.0,0.00,2,19,0,9.500000,2,1,25,32.000000,25.000000,25.00,32.000000,35.000000,9.700000,0.000000,1.400000,3.600000,9.0
4497,CH Morris,784,0,Kolkata Knight Riders,Sharjah Cricket Stadium,0,0,0,0.0,0,0,0,0.0,0.000000,0.0,0.0,0.00,2,17,0,4.250000,4,2,56,43.000000,43.090909,56.00,35.259259,27.698413,7.955026,0.000000,1.285714,3.523810,9.0
4498,Washington Sundar,784,0,Kolkata Knight Riders,Sharjah Cricket Stadium,0,0,0,0.0,0,0,0,0.0,0.000000,0.0,0.0,0.00,2,20,0,5.000000,4,3,85,22.000000,44.750000,85.00,27.461538,20.703704,7.540123,0.000000,0.851852,3.000000,9.0
4499,PP Chawla,785,0,Sunrisers Hyderabad,Dubai International Cricket Stadium,0,0,0,0.0,0,0,0,0.0,0.000000,0.0,0.0,0.00,2,8,0,8.000000,1,0,0,41.666667,22.642857,31.60,33.101449,26.117284,8.218107,0.012346,1.012346,3.351852,9.0


Combining bowling and Batting Points of the player



In [5]:
sum = df['total_points'] + df['bowling_fp']
df['total_fp_points'] = sum
df

,all_rounder,match_id,batted_for,opposition_team,venue,bat_inning,batsman_runs,balls,strike_rate,total_batting_points,bat_fp,total_points,batting_avg_inning_points,batting_avg_venue_points,batting_avg_oppo_points,avg_strike_rate,avg_batsman_runs,bowl_inning,total_bowler_runs,maiden,economy,overs,wickets,bowling_fp,bowl_fp_avg,bowl_opp_avg,bowl_ven_avg,bowl_inn_avg,bowl_run_avg,eco_avg,maiden_avg,wicket_avg,bowl_over_avg,rating,total_fp_points
0,A Ashish Reddy,286,Deccan Chargers,Mumbai Indians,Wankhede Stadium,1,10,10,100.0,7,0,7,7.0,7.000000,7.0,100.0,10.00,2,11,0,3.666667,3,1,31,58.000000,31.000000,31.00,44.500000,21.500000,5.833333,0.000000,1.500000,3.500000,8.0,38
1,A Ashish Reddy,292,Deccan Chargers,Chennai Super Kings,"MA Chidambaram Stadium, Chepauk",2,3,3,100.0,3,0,3,3.0,3.000000,3.0,100.0,6.50,1,16,0,8.000000,2,1,25,37.333333,25.000000,25.00,25.000000,22.750000,7.583333,0.000000,1.250000,3.000000,8.0,28
2,A Ashish Reddy,298,Deccan Chargers,Kings XI Punjab,"Rajiv Gandhi International Stadium, Uppal",2,8,8,100.0,5,0,5,4.0,5.000000,5.0,100.0,7.00,1,39,0,9.750000,4,2,72,27.000000,72.000000,72.00,48.500000,27.666667,8.180556,0.000000,1.333333,3.333333,8.0,77
3,A Ashish Reddy,314,Deccan Chargers,Rajasthan Royals,"Rajiv Gandhi International Stadium, Uppal",2,10,4,250.0,4,0,4,4.0,4.500000,4.0,137.5,7.75,1,17,0,8.500000,2,0,0,33.000000,0.000000,22.00,32.333333,26.500000,9.010417,0.000000,1.000000,3.000000,9.5,4
4,A Ashish Reddy,317,Deccan Chargers,Royal Challengers Bangalore,"Rajiv Gandhi International Stadium, Uppal",1,4,5,80.0,4,0,4,5.5,4.333333,4.0,126.0,7.00,2,25,0,6.250000,4,3,81,22.000000,57.000000,36.75,36.666667,26.333333,8.703704,0.000000,1.222222,3.111111,8.0,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496,I Udana,784,0,Kolkata Knight Riders,Sharjah Cricket Stadium,0,0,0,0.0,0,0,0,0.0,0.000000,0.0,0.0,0.00,2,19,0,9.500000,2,1,25,32.000000,25.000000,25.00,32.000000,35.000000,9.700000,0.000000,1.400000,3.600000,9.0,25
4497,CH Morris,784,0,Kolkata Knight Riders,Sharjah Cricket Stadium,0,0,0,0.0,0,0,0,0.0,0.000000,0.0,0.0,0.00,2,17,0,4.250000,4,2,56,43.000000,43.090909,56.00,35.259259,27.698413,7.955026,0.000000,1.285714,3.523810,9.0,56
4498,Washington Sundar,784,0,Kolkata Knight Riders,Sharjah Cricket Stadium,0,0,0,0.0,0,0,0,0.0,0.000000,0.0,0.0,0.00,2,20,0,5.000000,4,3,85,22.000000,44.750000,85.00,27.461538,20.703704,7.540123,0.000000,0.851852,3.000000,9.0,85
4499,PP Chawla,785,0,Sunrisers Hyderabad,Dubai International Cricket Stadium,0,0,0,0.0,0,0,0,0.0,0.000000,0.0,0.0,0.00,2,8,0,8.000000,1,0,0,41.666667,22.642857,31.60,33.101449,26.117284,8.218107,0.012346,1.012346,3.351852,9.0,0


Data Processing

In [6]:
df1 = df.drop(columns=['bat_inning', 'batsman_runs', 'balls', 'strike_rate', 'total_batting_points', 'bat_fp', 'total_points', 'bowl_inning', 'total_bowler_runs', 'maiden', 'economy', 'overs', 'wickets', 'bowling_fp'])
df1 

,all_rounder,match_id,batted_for,opposition_team,venue,batting_avg_inning_points,batting_avg_venue_points,batting_avg_oppo_points,avg_strike_rate,avg_batsman_runs,bowl_fp_avg,bowl_opp_avg,bowl_ven_avg,bowl_inn_avg,bowl_run_avg,eco_avg,maiden_avg,wicket_avg,bowl_over_avg,rating,total_fp_points
0,A Ashish Reddy,286,Deccan Chargers,Mumbai Indians,Wankhede Stadium,7.0,7.000000,7.0,100.0,10.00,58.000000,31.000000,31.00,44.500000,21.500000,5.833333,0.000000,1.500000,3.500000,8.0,38
1,A Ashish Reddy,292,Deccan Chargers,Chennai Super Kings,"MA Chidambaram Stadium, Chepauk",3.0,3.000000,3.0,100.0,6.50,37.333333,25.000000,25.00,25.000000,22.750000,7.583333,0.000000,1.250000,3.000000,8.0,28
2,A Ashish Reddy,298,Deccan Chargers,Kings XI Punjab,"Rajiv Gandhi International Stadium, Uppal",4.0,5.000000,5.0,100.0,7.00,27.000000,72.000000,72.00,48.500000,27.666667,8.180556,0.000000,1.333333,3.333333,8.0,77
3,A Ashish Reddy,314,Deccan Chargers,Rajasthan Royals,"Rajiv Gandhi International Stadium, Uppal",4.0,4.500000,4.0,137.5,7.75,33.000000,0.000000,22.00,32.333333,26.500000,9.010417,0.000000,1.000000,3.000000,9.5,4
4,A Ashish Reddy,317,Deccan Chargers,Royal Challengers Bangalore,"Rajiv Gandhi International Stadium, Uppal",5.5,4.333333,4.0,126.0,7.00,22.000000,57.000000,36.75,36.666667,26.333333,8.703704,0.000000,1.222222,3.111111,8.0,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496,I Udana,784,0,Kolkata Knight Riders,Sharjah Cricket Stadium,0.0,0.000000,0.0,0.0,0.00,32.000000,25.000000,25.00,32.000000,35.000000,9.700000,0.000000,1.400000,3.600000,9.0,25
4497,CH Morris,784,0,Kolkata Knight Riders,Sharjah Cricket Stadium,0.0,0.000000,0.0,0.0,0.00,43.000000,43.090909,56.00,35.259259,27.698413,7.955026,0.000000,1.285714,3.523810,9.0,56
4498,Washington Sundar,784,0,Kolkata Knight Riders,Sharjah Cricket Stadium,0.0,0.000000,0.0,0.0,0.00,22.000000,44.750000,85.00,27.461538,20.703704,7.540123,0.000000,0.851852,3.000000,9.0,85
4499,PP Chawla,785,0,Sunrisers Hyderabad,Dubai International Cricket Stadium,0.0,0.000000,0.0,0.0,0.00,41.666667,22.642857,31.60,33.101449,26.117284,8.218107,0.012346,1.012346,3.351852,9.0,0


Encoding Categorical Variables

In [8]:
df=df1
df["batted_for"]= df["batted_for"].astype("category")
df["Team_code"] = df["batted_for"].cat.codes

df["venue"]= df["venue"].astype("category")
df["venue_code"] = df["venue"].cat.codes

df["all_rounder"]= df["all_rounder"].astype("category")
df["all_rounder_code"] = df["all_rounder"].cat.codes

df['opposition_team'] = df['opposition_team'].astype("category")
df["opposition_team_code"] = df['opposition_team'].cat.codes

all_rounder_zip = dict(zip(df['all_rounder_code'],df['all_rounder']))
print(all_rounder_zip)

df_ready = df.drop(columns=["venue","all_rounder", "opposition_team"])
df_ready

{0: 'A Ashish Reddy', 1: 'A Chandila', 2: 'A Dananjaya', 3: 'A Flintoff', 4: 'A Symonds', 5: 'A Uniyal', 6: 'AA Bilakhia', 7: 'AA Noffke', 8: 'AB McDonald', 9: 'AD Mascarenhas', 10: 'AD Mathews', 11: 'AD Russell', 12: 'AL Menaria', 13: 'AM Nayar', 14: 'AP Majumdar', 15: 'AR Patel', 16: 'AS Raut', 18: 'Ankit Soni', 19: 'Azhar Mahmood', 20: 'BA Stokes', 21: 'BB Samantray', 22: 'BCJ Cutting', 23: 'BMAJ Mendis', 24: 'Bipul Sharma', 25: 'C de Grandhomme', 26: 'CH Gayle', 27: 'CH Morris', 28: 'CJ Anderson', 29: 'CR Brathwaite', 30: 'CR Woakes', 31: 'D Wiese', 32: 'D du Preez', 33: 'DJ Bravo', 34: 'DJ Hooda', 35: 'DJ Hussey', 37: 'DJG Sammy', 38: 'DL Vettori', 39: 'DR Smith', 40: 'DT Christian', 41: 'F Behardien', 42: 'GB Hogg', 43: 'GH Vihari', 44: 'GJ Maxwell', 45: 'GR Napier', 46: 'HH Pandya', 47: 'I Udana', 48: 'IK Pathan', 49: 'Iqbal Abdulla', 50: 'J Botha', 51: 'JA Morkel', 52: 'JD Ryder', 53: 'JDP Oram', 54: 'JDS Neesham', 55: 'JEC Franklin', 56: 'JH Kallis', 57: 'JJ van der Wath', 58:

,match_id,batted_for,batting_avg_inning_points,batting_avg_venue_points,batting_avg_oppo_points,avg_strike_rate,avg_batsman_runs,bowl_fp_avg,bowl_opp_avg,bowl_ven_avg,bowl_inn_avg,bowl_run_avg,eco_avg,maiden_avg,wicket_avg,bowl_over_avg,rating,total_fp_points,Team_code,venue_code,all_rounder_code,opposition_team_code
0,286,Deccan Chargers,7.0,7.000000,7.0,100.0,10.00,58.000000,31.000000,31.00,44.500000,21.500000,5.833333,0.000000,1.500000,3.500000,8.0,38,2,35,0,8
1,292,Deccan Chargers,3.0,3.000000,3.0,100.0,6.50,37.333333,25.000000,25.00,25.000000,22.750000,7.583333,0.000000,1.250000,3.000000,8.0,28,2,16,0,0
2,298,Deccan Chargers,4.0,5.000000,5.0,100.0,7.00,27.000000,72.000000,72.00,48.500000,27.666667,8.180556,0.000000,1.333333,3.333333,8.0,77,2,24,0,5
3,314,Deccan Chargers,4.0,4.500000,4.0,137.5,7.75,33.000000,0.000000,22.00,32.333333,26.500000,9.010417,0.000000,1.000000,3.000000,9.5,4,2,24,0,10
4,317,Deccan Chargers,5.5,4.333333,4.0,126.0,7.00,22.000000,57.000000,36.75,36.666667,26.333333,8.703704,0.000000,1.222222,3.111111,8.0,85,2,24,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496,784,0,0.0,0.000000,0.0,0.0,0.00,32.000000,25.000000,25.00,32.000000,35.000000,9.700000,0.000000,1.400000,3.600000,9.0,25,0,29,47,7
4497,784,0,0.0,0.000000,0.0,0.0,0.00,43.000000,43.090909,56.00,35.259259,27.698413,7.955026,0.000000,1.285714,3.523810,9.0,56,0,29,27,7
4498,784,0,0.0,0.000000,0.0,0.0,0.00,22.000000,44.750000,85.00,27.461538,20.703704,7.540123,0.000000,0.851852,3.000000,9.0,85,0,29,134,7
4499,785,0,0.0,0.000000,0.0,0.0,0.00,41.666667,22.642857,31.60,33.101449,26.117284,8.218107,0.012346,1.012346,3.351852,9.0,0,0,6,90,14


Selecting Relevant Features

In [9]:
df2 = df[['match_id', 'batting_avg_inning_points', 'batting_avg_venue_points', 'batting_avg_oppo_points', 'avg_strike_rate', 'avg_batsman_runs', 'bowl_fp_avg', 'bowl_opp_avg', 'bowl_ven_avg', 'bowl_inn_avg', 'bowl_run_avg', 'eco_avg', 'maiden_avg', 'wicket_avg', 'bowl_over_avg', 'Team_code', 'venue_code', 'all_rounder_code', 'opposition_team_code']]
df2

,match_id,batting_avg_inning_points,batting_avg_venue_points,batting_avg_oppo_points,avg_strike_rate,avg_batsman_runs,bowl_fp_avg,bowl_opp_avg,bowl_ven_avg,bowl_inn_avg,bowl_run_avg,eco_avg,maiden_avg,wicket_avg,bowl_over_avg,Team_code,venue_code,all_rounder_code,opposition_team_code
0,286,7.0,7.000000,7.0,100.0,10.00,58.000000,31.000000,31.00,44.500000,21.500000,5.833333,0.000000,1.500000,3.500000,2,35,0,8
1,292,3.0,3.000000,3.0,100.0,6.50,37.333333,25.000000,25.00,25.000000,22.750000,7.583333,0.000000,1.250000,3.000000,2,16,0,0
2,298,4.0,5.000000,5.0,100.0,7.00,27.000000,72.000000,72.00,48.500000,27.666667,8.180556,0.000000,1.333333,3.333333,2,24,0,5
3,314,4.0,4.500000,4.0,137.5,7.75,33.000000,0.000000,22.00,32.333333,26.500000,9.010417,0.000000,1.000000,3.000000,2,24,0,10
4,317,5.5,4.333333,4.0,126.0,7.00,22.000000,57.000000,36.75,36.666667,26.333333,8.703704,0.000000,1.222222,3.111111,2,24,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496,784,0.0,0.000000,0.0,0.0,0.00,32.000000,25.000000,25.00,32.000000,35.000000,9.700000,0.000000,1.400000,3.600000,0,29,47,7
4497,784,0.0,0.000000,0.0,0.0,0.00,43.000000,43.090909,56.00,35.259259,27.698413,7.955026,0.000000,1.285714,3.523810,0,29,27,7
4498,784,0.0,0.000000,0.0,0.0,0.00,22.000000,44.750000,85.00,27.461538,20.703704,7.540123,0.000000,0.851852,3.000000,0,29,134,7
4499,785,0.0,0.000000,0.0,0.0,0.00,41.666667,22.642857,31.60,33.101449,26.117284,8.218107,0.012346,1.012346,3.351852,0,6,90,14


Normalizing the features

In [10]:
X = df2.values
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X

array([[-0.51668816, -0.56684813, -0.45367151, ...,  1.73027174,
        -1.83761108,  0.21722878],
       [-0.48938042, -0.8323543 , -0.66727668, ..., -0.19158073,
        -1.83761108, -1.59972215],
       [-0.46207269, -0.76597776, -0.5604741 , ...,  0.61762031,
        -1.83761108, -0.46412782],
       ...,
       [ 1.74985407, -1.03148393, -0.82748056, ...,  1.12337096,
         1.70919013, -0.00989009],
       [ 1.75440536, -1.03148393, -0.82748056, ..., -1.20308203,
         0.54456883,  1.57994197],
       [ 1.75895665, -1.03148393, -0.82748056, ..., -1.20308203,
         0.59750617,  0.67146651]])

In [11]:
y = df['total_fp_points'].values
y

array([38, 28, 77, ..., 85,  0, 31])

Data Splitting

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (3600, 19) (3600,)
Test set: (901, 19) (901,)


Random Forest Model 

In [ ]:
model_rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=100)
model_rf.fit(X_train, y_train) 
pred_train_rf= model_rf.predict(X_train)
print('Training Accuracy:')
print(np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print(r2_score(y_train, pred_train_rf))

print()
pred_test_rf = model_rf.predict(X_test)
print('Test Accuracy:')
print(np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print(r2_score(y_test, pred_test_rf))

Training Accuracy:
11.829291331535464
0.9452300338443084

Test Accuracy:
28.736087353749102
0.6579892287900265


In [ ]:
x=X
y=model_rf.predict(x)
y

array([  8.638,  53.76 , 105.836, ...,  21.242,  79.574,   2.7  ])

K-Fold Cross validation to test Generalization

In [ ]:
scores = cross_val_score(model_rf, X, y, cv=5)
print(scores)

[0.67288428 0.62254463 0.69318134 0.687168   0.62236258]


Random Forest Model Hyper Parameter Tuning

In [ ]:
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(model_rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 500,
 'n_jobs': None,
 'oob_score': True,
 'random_state': 100,
 'verbose': 0,
 'warm_start': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
rf_random = RandomizedSearchCV(estimator = model_rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   57.8s
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed:  7.9min
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 21.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=500,
                              

In [ ]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 600}

In [ ]:
pred_train_rf= rf_random.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print(r2_score(y_train, pred_train_rf))

pred_test_rf = rf_random.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print(r2_score(y_test, pred_test_rf))

9.719022882226376
0.9406554791036668
21.56373556952009
0.7115397018733759


Finding the prediction after tuning the model

In [ ]:
y_pred = rf_random.predict(X_train)
print("Predicted Respons: ", y_pred, sep = '\n')
y_pred1 = rf_random.predict(X_test)

Predicted Respons: 
[ 34.76297547  16.22111241  12.74381103 ...  85.57682038 125.05481788
  43.66094535]
